<a href="https://colab.research.google.com/github/YeongRoYun/BearTeam/blob/yyr-dev-data-eda/data/eda/EDA01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-HUB Dataset EDA
> ` https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=629`


In [ ]:
import os
basePath = os.getcwd()
dataPath = basePath + '/data'

if not os.path.exists(dataPath):
  os.makedirs(dataPath, exist_ok=True)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

## 파일구조 확인하기
- Raw Image와 동일한 폴더 구조에 각 Image의 Annotation JSON File이 위치한다.
- File Structure

```
1.Training/2.Validation
   |
   | ---- Annotations(라벨링데이터)/Raws(원천데이터)
                |
                | ---- BoundingBox(...2D BB)
                          |
                          | --- 1.Day(주간)/2.Night(야간)
                                    |
                                    | ---- 1.Sun(맑음)/2.Rain(비)/3.Snow(눈)
                                                    |
                                                    | ---- 1. OldTown(구도심)
                                                    | ---- 2. NewTown(신도심)
                                                    | ---- 3. Alley(골목)
                                                    | ---- 4. Country(시골길)
                                                    | ---- 5. Mountain(산길)
                                                    | ---- 6. Coastal(해안도로)
                                                    | ---- 7. Highway(고속도로)
                                                    | ---- 8. Motorway(자동차전용)
                                                        |
                                                        | dates(yyyymmdd_hhmm)
                                                            |-- Camera
                                                              |
                                                              | -- front
                                                              | -- left
                                                              | -- rear
                                                              | -- right
                                                              | -- stereo_l
                                                              | -- stereo_r
```
- 데이터셋은 위의 파일 구조 형태를 공유한다.

## 파일구조 통일시키기
- 한글과 영어가 혼용된 파일명을 영어로 일관되게 통일시킨다.
- 예시: `주간 -> Day, 맑음 -> Sun`

> Annotation data로 Formating을 검증한다.

In [ ]:
import zipfile
zipPath = dataPath + '/dataset.zip'

datasetZip = zipfile.ZipExtFile(zipPath)